In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_style('whitegrid')

In [23]:
df_test_features = pd.read_csv('../test_features.csv')

In [24]:
num_test_data = df_test_features.iloc[:,9:]

In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [26]:
num_test = scaler.fit_transform(num_test_data)

In [27]:
import ktrain

In [28]:
predictor = ktrain.load_predictor('../audio_text_model')

# Pre-process string with ktrain

In [29]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [30]:
# remove stopwords
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [31]:
x_test2 = df_test_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
x_test2 = x_test2.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
x_test2 = np.array(x_test2)

In [32]:
x_text_input = predictor.preproc.preprocess_test(x_test2)

428 test sequences
test sequence lengths:
	mean : 54
	95percentile : 109
	99percentile : 150
x_test shape: (428,150)


# Make a prediction

In [33]:
test_dataset = [num_test] + [x_text_input[0]]

In [34]:
prediction = predictor.model.predict(test_dataset)

In [35]:
predict_labels = [ label.argmax() for label in prediction]

In [36]:
df = pd.DataFrame(predict_labels, columns=['genre_pred'])

In [37]:
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}

In [38]:
df['genre_pred'] = df['genre_pred'].map(classes_mapping)

In [39]:
df['trackID'] = df_test_features['trackID']

In [40]:
correct_label = pd.read_csv('../test_labels.csv')

In [41]:
# merge df and correct_label by trackID
df_merged = pd.merge(df, correct_label, on='trackID')

In [42]:
from sklearn.metrics import classification_report
print(classification_report(df_merged['genre'], df_merged['genre_pred']))

                       precision    recall  f1-score   support

 classic pop and rock       0.30      0.41      0.34        64
dance and electronica       0.54      0.44      0.48        64
                 folk       0.52      0.68      0.59        73
       jazz and blues       0.12      0.08      0.10        36
                metal       0.95      0.50      0.66        42
                  pop       0.75      0.98      0.85        44
                 punk       0.56      0.50      0.53        40
      soul and reggae       0.54      0.43      0.48        65

             accuracy                           0.51       428
            macro avg       0.53      0.50      0.50       428
         weighted avg       0.53      0.51      0.51       428

